In [71]:
import requests
import pandas as pd
from pycoingecko import CoinGeckoAPI
import time
import datetime
import psycopg2
import datetime
now = datetime.datetime.now()

In [88]:
print(now)

2020-10-09 19:51:59.877153


In [2]:
cg = CoinGeckoAPI()

In [3]:
#Connect the dbname to the name of the database we create
conn = psycopg2.connect("host=localhost dbname='task2Playground' user='postgres' password='Runner48'")
cur = conn.cursor()


In [5]:
#creates a lists for naming each table
coins = ['ethereum', 'yearnFinance', 'chainlink', 'ethlend', 'uniswap', 'havven', 'uma', 'nxm', 'rarible']
coin = ['ethereum', 'yearn-finance', 'chainlink', 'ethlend', 'uniswap', 'havven', 'uma', 'nxm', 'rarible']

In [6]:
#Create each table in the databse
for i in range(len(coins)):
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {coins[i]} (
    coin_id SERIAL PRIMARY KEY,
    date TIMESTAMP NOT NULL,
    price decimal NOT NULL
)
             """)

In [7]:
conn.commit()

In [94]:
#while True:
token_prices = cg.get_price(ids=coin, vs_currencies='usd')
for x in range(len(coins)):
    name = coin[x]
    coin_price = float(token_prices[coin[x]]['usd'])
    timeDate = now.strftime("%Y-%m-%d %H:%M:%S")
    cur.execute(f"INSERT INTO {coins[x]} (date, price) VALUES (TIMESTAMP'{timeDate}',{coin_price})")
    conn.commit()
   